In [1]:
import torch as tr
import su2_chain as s
import integrators as integ

In [2]:
a = tr.normal(0.0,2.0,[4])
a = 2*a/tr.norm(a)
print(a)
U = s.Mat(a) #tr.sum(a[:,None,None] * basis,dim=0)
print(U)
print("Should be identity")
print(s.TdxT(U,U))

tensor([ 0.3587, -1.7242,  0.9467, -0.0465])
tensor([[ 0.1794-0.0233j, -0.4734-0.8621j],
        [ 0.4734-0.8621j,  0.1794+0.0233j]])
Should be identity
tensor([[1.0000+1.1954e-08j, 0.0000+0.0000e+00j],
        [0.0000+0.0000e+00j, 1.0000-1.1954e-08j]])


In [3]:
b = tr.normal(0.0,2.0,[4])
b = 2*b/tr.norm(b)
print(b)
V = s.Mat(b) #tr.sum(a[:,None,None] * basis,dim=0)
print(V)
print("Should be identity")
print(s.TdxT(V,V))

tensor([ 0.7586, -1.5493, -0.5290, -0.8627])
tensor([[ 0.3793-0.4314j,  0.2645-0.7747j],
        [-0.2645-0.7747j,  0.3793+0.4314j]])
Should be identity
tensor([[1.0000+2.0226e-09j, 0.0000+0.0000e+00j],
        [0.0000+0.0000e+00j, 1.0000-2.0226e-09j]])


In [4]:
tr.trace(U@tr.conj(V.T)).real.item()/2-(a@b/4).item()

5.960464477539063e-08

In [5]:
su2f = s.field([128,128],Nbatch=2)

In [6]:
su2c = s.SU2chain(beta=2.0,field_type=su2f)

In [7]:
P = su2c.refreshP()

In [8]:
U =su2c.f.hot()
U.shape

torch.Size([2, 128, 128, 4])


torch.Size([2, 128, 128, 2, 2])

In [9]:
a= su2c.f.coeffs(U)

In [10]:
a.shape

torch.Size([2, 128, 128, 4])

In [11]:
s.Mat(a[0,0,0,:])- U[0,0,0,:,:]

tensor([[0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j]])

In [12]:
FF = su2c.f.mat(a)

In [13]:
(FF-U).norm()

tensor(0.)

In [14]:
P.shape

torch.Size([2, 128, 128, 2, 2])

In [15]:
P[...,1,0].real.mean(), P[...,1,0].real.std(),P[...,1,0].imag.mean(), P[...,1,0].imag.std()

(tensor(-0.0006), tensor(0.5009), tensor(0.0008), tensor(0.5003))

In [16]:
P[...,0,0].real.mean(), P[...,0,0].real.std(),P[...,0,0].imag.mean(), P[...,0,0].imag.std()

(tensor(0.), tensor(0.), tensor(-0.0013), tensor(0.5009))

In [17]:
tr.sqrt(tr.tensor(2))

tensor(1.4142)

In [18]:
pp = 2.0*tr.einsum('kij,...ji->...k',s.tau,P)

In [19]:
pp.real.std(dim=(0,1,2))

tensor([1.0005, 1.0018, 1.0017])

In [20]:
pp.shape

torch.Size([2, 128, 128, 3])

In [21]:
ff = -tr.einsum('...k,kij->...ij',pp,s.tau)

In [22]:
ff[0,0,0]-P[0,0,0]

tensor([[-0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j]])

In [23]:
tr.trace(P[0,0,0])

tensor(0.+0.j)

In [24]:
tr.einsum('...ij,...ij->',P,P.conj()).real

tensor(49283.2422)

In [25]:
tr.einsum('...k,...k->',pp,pp)/2

tensor(49283.2109+0.j)

In [26]:
(tr.norm(P,dim=(su2c.f.ci[0],su2c.f.ci[1])) - tr.norm(pp,dim=(len(pp.shape)-1))/tr.sqrt(tr.tensor(2))).norm()

tensor(1.2442e-05)

In [27]:
E = su2c.f.expo(P)

In [28]:
E[0,0,0] @ E[0,0,0].conj().transpose(0,1)

tensor([[ 1.0000e+00+0.j, -3.2249e-10+0.j],
        [-3.2249e-10+0.j,  1.0000e+00+0.j]])

In [29]:
tr.eye(2,2) + P[0,0,0]@(tr.eye(2,2) + 0.5*P[0,0,0]@(tr.eye(2,2)+ 1/3*P[0,0,0]@(tr.eye(2,2) + 1/4*P[0,0,0]))), E[0,0,0]

(tensor([[ 0.4029+0.5574j,  0.3409-0.6186j],
         [-0.3409-0.6186j,  0.4029-0.5574j]]),
 tensor([[ 0.3997+0.5679j,  0.3473-0.6302j],
         [-0.3473-0.6302j,  0.3997-0.5679j]]))

In [30]:
P = su2c.refreshP()
Ken = su2c.kinetic(P)
Ven = su2c.action(U)
Hi = Ken + Ven
print("The total initial kinetic energy is: ",Ken)
print("The total initial potential energy is: ",Ven)
print("The total initial energy is: ",Hi)

The total initial kinetic energy is:  tensor([24678.9434, 24559.0566])
The total initial potential energy is:  tensor([65234.5312, 65500.9297])
The total initial energy is:  tensor([89913.4766, 90059.9844])


In [31]:
dt = 0.1
k = 10
l = integ.leapfrog(su2c.force,su2f.evolveQ,k,1.0)
m = integ.minnorm2(su2c.force,su2f.evolveQ,k,1.0)
PP ,QQ  = l.integrate(P,U)
PP2,QQ2 = m.integrate(P,U)

In [32]:
KenF = su2c.kinetic(PP)
VenF = su2c.action(QQ)
Hf = KenF + VenF
print("The total initial kinetic energy is: ",KenF)
print("The total initial potential energy is: ",VenF)
print("The total initial energy is: ",Hf)
print("The total energy diff is: ",Hf-Hi)

The total initial kinetic energy is:  tensor([63384.7227, 63153.3750])
The total initial potential energy is:  tensor([80647.8047, 80779.1562])
The total initial energy is:  tensor([144032.5312, 143932.5312])
The total energy diff is:  tensor([54119.0547, 53872.5469])


In [33]:
KenF = su2c.kinetic(PP2)
VenF = su2c.action(QQ2)
Hf = KenF + VenF
print("The total initial kinetic energy is: ",KenF)
print("The total initial potential energy is: ",VenF)
print("The total initial energy is: ",Hf)
print("The total energy diff is: ",Hf-Hi)

The total initial kinetic energy is:  tensor([63513.3203, 63281.7227])
The total initial potential energy is:  tensor([80689.8516, 80821.3906])
The total initial energy is:  tensor([144203.1719, 144103.1094])
The total energy diff is:  tensor([54289.6953, 54043.1250])


In [39]:
foo = tr.einsum('kij,klm->ijlm',s.tau,s.tau)

In [52]:
foo[0,0,1,1]

tensor(0.2500+0.j)

In [53]:
U

tensor([[[[[ 0.8563-0.5045j, -0.0079+0.1100j],
           [ 0.0079+0.1100j,  0.8563+0.5045j]],

          [[ 0.6737+0.0687j, -0.0706+0.7324j],
           [ 0.0706+0.7324j,  0.6737-0.0687j]],

          [[ 0.5979-0.7358j,  0.3102+0.0701j],
           [-0.3102+0.0701j,  0.5979+0.7358j]],

          ...,

          [[-0.4399-0.4763j,  0.6419+0.4093j],
           [-0.6419+0.4093j, -0.4399+0.4763j]],

          [[-0.6025-0.0933j,  0.4669-0.6406j],
           [-0.4669-0.6406j, -0.6025+0.0933j]],

          [[ 0.0201+0.0138j,  0.9656-0.2590j],
           [-0.9656-0.2590j,  0.0201-0.0138j]]],


         [[[-0.1203-0.3299j, -0.1690+0.9210j],
           [ 0.1690+0.9210j, -0.1203+0.3299j]],

          [[ 0.9226-0.0844j,  0.2718-0.2602j],
           [-0.2718-0.2602j,  0.9226+0.0844j]],

          [[-0.9321-0.0189j, -0.3318-0.1440j],
           [ 0.3318-0.1440j, -0.9321+0.0189j]],

          ...,

          [[-0.0930-0.5381j,  0.3804+0.7464j],
           [-0.3804+0.7464j, -0.0930+0.5381j]],

      

In [55]:
tr.trace(U[0,0,0]@U[0,0,1].T.conj())

tensor(1.2466+0.j)